**Displaying the columns and its datatypes**

In [1]:
import pandas as pd
first_five = pd.read_csv('Data/crunchbase-investments.csv',  nrows=5)
first_five.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   company_permalink       5 non-null      object
 1   company_name            5 non-null      object
 2   company_category_code   5 non-null      object
 3   company_country_code    5 non-null      object
 4   company_state_code      4 non-null      object
 5   company_region          5 non-null      object
 6   company_city            4 non-null      object
 7   investor_permalink      5 non-null      object
 8   investor_name           5 non-null      object
 9   investor_category_code  4 non-null      object
 10  investor_country_code   5 non-null      object
 11  investor_state_code     5 non-null      object
 12  investor_region         5 non-null      object
 13  investor_city           5 non-null      object
 14  funding_round_type      5 non-null      object
 15  funded_at 

**Displaying data**

In [2]:
first_five

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000


**Reading data in chunks to analyze and optimize memory consumption**

In [4]:
chunk_iter = pd.read_csv('Data/crunchbase-investments.csv', chunksize=5000, encoding='Latin-1')

col_missing_values,col_mem_footprint,col_unique_count = [], [], {}
total_mem_consumption = 0

for chunk in chunk_iter:
    # Collecting missing value count of each column in dataframe chunk 
    col_missing_values.append(chunk.isnull().sum(axis=0))
    
    # Collecting memory consumption of each dataframe chunk
    total_mem_consumption += chunk.memory_usage(deep=True).sum() / 2**20
    col_mem_footprint.append(chunk.memory_usage(deep=True))
    
    #Collecting unique value count of each column of type object of the dataframe chunk
    object_cols = chunk.select_dtypes(include='object')
    for col in object_cols.columns:
        if col not in col_unique_count:
            col_unique_count[col] = []
        col_unique_count[col].append(chunk[col].value_counts())
        
print(f'Total memory consumed by the whole dataframe {total_mem_consumption:.2f} MB' )

Total memory consumed by the whole dataframe 56.99 MB


**Checking for columns without missing values for conversion to integer datatype**

In [5]:
# Grouping missing value count of all columns across all dataframes 
missing_val_series = pd.concat(col_missing_values)
missing_val_series.groupby(missing_val_series.index).sum()

company_category_code       643
company_city                533
company_country_code          1
company_name                  1
company_permalink             1
company_region                1
company_state_code          492
funded_at                     3
funded_month                  3
funded_quarter                3
funded_year                   3
funding_round_type            3
investor_category_code    50427
investor_city             12480
investor_country_code     12001
investor_name                 2
investor_permalink            2
investor_region               2
investor_state_code       16809
raised_amount_usd          3599
dtype: int64

**Observation: As all columns have missing values, no column can be considered for conversion to integer data type**

**Analyzing memory footprint of each column to check which columns have scope for memory optimization**

In [6]:
concat = pd.concat(col_mem_footprint)
concat.groupby(concat.index).sum() / 2**20

Index                     0.001381
company_category_code     3.262619
company_city              3.343473
company_country_code      3.025223
company_name              3.424955
company_permalink         3.869808
company_region            3.253503
company_state_code        2.962161
funded_at                 3.378091
funded_month              3.226837
funded_quarter            3.226837
funded_year               0.403366
funding_round_type        3.252704
investor_category_code    0.593590
investor_city             2.751430
investor_country_code     2.524654
investor_name             3.734270
investor_permalink        4.749821
investor_region           3.238946
investor_state_code       2.361876
raised_amount_usd         0.403366
dtype: float64

**Columns of type object that can be converted to type category** 

In [7]:
# Columns that have less than 50% unique values can be converted to type category 
for col in col_unique_count:
    combined_series = pd.concat(col_unique_count[col])
    combined_series = combined_series.groupby(combined_series.index).sum()
    if len(combined_series) / combined_series.sum() < 0.5:
        print(col)

company_permalink
company_name
company_category_code
company_country_code
company_state_code
company_region
company_city
investor_permalink
investor_name
investor_category_code
investor_country_code
investor_state_code
investor_region
investor_city
funding_round_type
funded_at
funded_month
funded_quarter


**Minimizing the memory footprint of each dataframe chunk using the above analysis and loading that data into an sqlite table for further querying and analysis**

In [9]:
import sqlite3
import numpy as np
sqlite3.register_adapter(np.int64, lambda val: int(val))
conn = sqlite3.connect('Data/crunchbase.db')
cur=conn.cursor()

In [10]:
cur.execute('DROP TABLE INVESTMENTS')
categories_dict = {
  'company_name': 'category',
   'company_category_code' : 'category', 
   'investor_name' : 'category', 
   'investor_category_code' : 'category', 
   'funding_round_type' : 'category', 
}
chunk_iter = pd.read_csv('Data/crunchbase-investments.csv', chunksize=5000, usecols=['company_name', 'company_category_code', 
                                                                                'investor_name','investor_category_code',
                                                                                'funding_round_type', 'funded_at', 'raised_amount_usd'],
                                            parse_dates=['funded_at'], dtype=categories_dict, encoding='Latin-1')
total_optimized_mem = 0 
for chunk in chunk_iter:
    chunk.to_sql('INVESTMENTS',conn,if_exists='append',index=False, dtype={'raised_amount_usd': 'INTEGER'})
    total_optimized_mem += chunk.memory_usage(deep=True).sum() / 2**20
    
print(f'Total memory consumed by the whole dataframe {total_optimized_mem:.2f} MB' )

Total memory consumed by the whole dataframe 5.16 MB


**Checking the schema of sqlite table *INVESTMENTS***

In [11]:
results_df = pd.read_sql("PRAGMA table_info(INVESTMENTS);", conn)
print(results_df)

   cid                    name       type  notnull dflt_value  pk
0    0            company_name       TEXT        0       None   0
1    1   company_category_code       TEXT        0       None   0
2    2           investor_name       TEXT        0       None   0
3    3  investor_category_code       TEXT        0       None   0
4    4      funding_round_type       TEXT        0       None   0
5    5               funded_at  TIMESTAMP        0       None   0
6    6       raised_amount_usd    INTEGER        0       None   0


**Which company raised the most money?**

In [12]:
query = """
        SELECT company_category_code,
        SUM(raised_amount_usd) as raised
        from INVESTMENTS
        GROUP BY company_category_code
        ORDER BY raised DESC
        LIMIT 1
        """
max_funding_cat = pd.read_sql(query, conn)
max_funding_cat

,company_category_code,raised
0,biotech,110396423062


**Which investor invested the most money?**

In [13]:
query = """
        SELECT investor_name,
        SUM(raised_amount_usd) as amt_invested
        from INVESTMENTS
        GROUP BY investor_name
        ORDER BY amt_invested DESC
        LIMIT 1
        """
max_funding_investor = pd.read_sql(query, conn)
max_funding_investor

,investor_name,amt_invested
0,Kleiner Perkins Caufield & Byers,11217826376


**Which investor invested the most money per startup?**

In [14]:
query = """
        SELECT investor_name,
        SUM(raised_amount_usd)/COUNT(raised_amount_usd) as avg_investment
        FROM investments
        GROUP BY investor_name
        ORDER BY avg_investment DESC
        LIMIT 1
        """
max_avg_funding_investor = pd.read_sql(query, conn)
max_avg_funding_investor

,investor_name,avg_investment
0,Marlin Equity Partners,2600000000


**Which was the most popular ad least popular funding round?**

In [15]:
query = """
        SELECT funding_round_type,
        COUNT(*) as frequency
        FROM investments
        WHERE funding_round_type IS NOT NULL
        GROUP BY funding_round_type
        ORDER BY frequency DESC
        """
founding_round_type_popularity = pd.read_sql(query, conn)
most_popular = founding_round_type_popularity.iloc[0][0]
least_popular = founding_round_type_popularity.iloc[len(founding_round_type_popularity) - 1][0]
most_popular, least_popular
print(f'The most popular funding round type was: {most_popular}')
print(f'The least popular funding round type was: {least_popular}')

The most popular funding round type was: series-a
The least popular funding round type was: crowdfunding


**What amount of funds did the top 10% raise?**

In [16]:
query="""
      select iv.company_name,
      cast(sum(raised_amount_usd) as double)/(select cast(sum(raised_amount_usd) as bigint) from INVESTMENTS) as percentage_funding,
      cast(sum(raised_amount_usd) as bigint) as funding_amount
      from INVESTMENTS as iv
      group by iv.company_name 
      order by funding_amount desc
      limit (select cast(count(distinct company_name)*.1 as int) from INVESTMENTS)
      """
top_10_percent = pd.read_sql(query, conn)
total_portion_of_investment = top_10_percent['funding_amount'].sum()
print(f'The top 10% raised {total_portion_of_investment / 1e9:.2f} billion dollars')

The top 10% raised 457.63 billion dollars


**What amount of funds did the top 1% raise?**

In [17]:
query="""
      select iv.company_name,
      cast(sum(raised_amount_usd) as double)/(select cast(sum(raised_amount_usd) as bigint) from INVESTMENTS) as percentage_funding,
      cast(sum(raised_amount_usd) as bigint) as funding_amount
      from INVESTMENTS as iv
      group by iv.company_name 
      order by funding_amount desc
      limit (select cast(count(distinct company_name)*.01 as int) from INVESTMENTS)
      """
top_1_percent = pd.read_sql(query, conn)
total_portion_of_investment = top_1_percent['funding_amount'].sum()
print(f'The top 1% raised {total_portion_of_investment / 1e9:.2f} billion dollars')

The top 1% raised 178.73 billion dollars
